# Executive Brief: Support Operations & SLA Optimization
**Prepared By**: Senior Data Analyst

## 1. The Business Problem
Our Support Operations team is facing challenges with inconsistent resolution times and missed SLAs. To address this, we have initiated a comprehensive audit of our ticket data to answer:
1. **Where are we failing?** (Descriptive Analytics)
2. **Why are we failing?** (Statistical & Root Cause Analysis)
3. **How can we fix it?** (Predictive Modeling & Strategic Recommendations)

### Core KPIs Audited
- **SLA Breach Rate**: Target < 10% for Critical Tickets.
- **Resolution Time**: Identifying barriers to speed.
- **Financial Risk**: Quantifying the cost of service failures.

In [ ]:
# 1. Setup & Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2_contingency, ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.cluster import KMeans

# Settings for cleaner output
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded.")

## 2. Load Data
Ingesting the raw ticket logs for analysis.

In [ ]:
# Load the dataset
try:
    df = pd.read_csv('../data/customer_support_tickets.csv')
    print(f"Dataset loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    # Fallback for different working directories
    try:
        df = pd.read_csv('customer_support_tickets.csv')
        print(f"Dataset loaded successfully. Shape: {df.shape}")
    except FileNotFoundError:
        print("Error: customer_support_tickets.csv not found.")

if 'df' in locals():
    display(df.head())

## 3. SLA Definition & Business Logic (Canonical)
**SINGLE SOURCE OF TRUTH**
Here we define exactly what constitutes a "Breach" and the financial cost associated with it.
Any downstream analysis MUST use `Resolution_Hours` and `Is_SLA_Breach` defined here.

**Logic Rules**:
1. **Ticket Creation**: Imputed (1-5h before first response) due to missing raw log.
2. **Resolution Hours**: `Time Resolved` - `Creation Time`.
3. **SLA Targets**: Critical (4h), High (8h), Normal (24h), Low (72h).

In [ ]:
# --- CANONICAL SLA LOGIC ENGINE ---

# A. Date Conversion
df['Time_Resolved'] = pd.to_datetime(df['Time to Resolution'], errors='coerce')
df['Time_First_Response'] = pd.to_datetime(df['First Response Time'], errors='coerce')

# B. Filter Valid Rows
df_sla = df.dropna(subset=['Time_Resolved', 'Time_First_Response']).copy()

# C. Impute Creation Date (Simulation of Ground Truth)
np.random.seed(42)
random_hours = pd.to_timedelta(np.random.randint(1, 6, size=len(df_sla)), unit='h')
df_sla['Ticket Creation Date'] = df_sla['Time_First_Response'] - random_hours

# D. Calculate Resolution Hours
df_sla['Resolution_Hours'] = (df_sla['Time_Resolved'] - df_sla['Ticket Creation Date']).dt.total_seconds() / 3600
df_sla = df_sla[df_sla['Resolution_Hours'] > 0].copy() # Filter hygiene

# E. Define SLA Targets
def get_sla_target(priority):
    targets = {'Critical': 4, 'High': 8, 'Normal': 24, 'Low': 72}
    return targets.get(priority, 24)

df_sla['SLA_Target_Hours'] = df_sla['Ticket Priority'].apply(get_sla_target)

# F. Determine Breach Status
df_sla['Is_SLA_Breach'] = df_sla['Resolution_Hours'] > df_sla['SLA_Target_Hours']
df_sla['Is_SLA_Breach_Numeric'] = df_sla['Is_SLA_Breach'].astype(int)

# G. Assign Financial Risk (Cost Logic)
def get_breach_cost(row):
    if not row['Is_SLA_Breach']: return 0
    # Cost = Penalty + Churn Risk Estimate
    costs = {'Critical': 500, 'High': 200, 'Normal': 50, 'Low': 10}
    return costs.get(row['Ticket Priority'], 0)

df_sla['Est_Breach_Cost'] = df_sla.apply(get_breach_cost, axis=1)

# Extract Hour for Workload Analyis
df_sla['Hour_of_Day'] = df_sla['Ticket Creation Date'].dt.hour

print("✅ SLA Logic & Financial Risk Engine Applied.")
print(f"Analyzable Dataset: {df_sla.shape[0]} tickets.")
display(df_sla[['Ticket Creation Date', 'Resolution_Hours', 'SLA_Target_Hours', 'Is_SLA_Breach', 'Est_Breach_Cost']].head())

## 4. Feature Engineering
Preparing the data for Machine Learning. We encode categorical variables and define the feature set.

In [ ]:
# Select Features for Prediction
features = ['Ticket Priority', 'Ticket Channel', 'Ticket Type', 'Customer Age']
target = 'Is_SLA_Breach_Numeric'

# Prepare ML Dataset
ml_df = df_sla[features + [target]].dropna().copy()

# One-Hot Encoding
ml_df = pd.get_dummies(ml_df, columns=['Ticket Priority', 'Ticket Channel', 'Ticket Type'], drop_first=True)

X = ml_df.drop(columns=[target])
y = ml_df[target]

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Features Prepared. Training Shape: {X_train.shape}")

## 5. Predictive Risk Modeling
Using **Random Forest** to predict breaches before they occur.

In [ ]:
# --- PHASE 4: PREDICTIVE RISK MODELING (SLA Breach) ---
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import average_precision_score

# Safety checks (prevents silent failures)
required_cols = ['Is_SLA_Breach', 'Ticket Type', 'Ticket Priority', 'Ticket Channel', 'Product Purchased']
missing = [c for c in required_cols if c not in df_sla.columns]
if missing:
    raise ValueError(f"Missing required columns before modeling: {missing}. "
                     "Run SLA feature engineering first (RPT_hours, SLA_Target_Hours, Is_SLA_Breach).")

# Features available at ticket creation (keep this “realistic” for ops triage)
feature_cols = ['Ticket Type', 'Ticket Priority', 'Ticket Channel', 'Product Purchased']
X = df_sla[feature_cols].copy()
y = df_sla['Is_SLA_Breach'].astype(int)

# Train/test split (stratify for imbalance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Preprocess categoricals
categorical = feature_cols
preprocess = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical)]
)

# Models (baseline + stronger)
models = {
    "LogReg": LogisticRegression(max_iter=2000, class_weight="balanced"),
    "RandomForest": RandomForestClassifier(
        n_estimators=400, random_state=42, class_weight="balanced_subsample"
    )
}

results = []
fitted = {}

for name, clf in models.items():
    pipe = Pipeline(steps=[("prep", preprocess), ("model", clf)])
    pipe.fit(X_train, y_train)
    fitted[name] = pipe
    
    # Probabilities for AUC metrics + thresholding
    p = pipe.predict_proba(X_test)[:, 1]
    yhat = (p >= 0.5).astype(int)

    roc = roc_auc_score(y_test, p)
    pr  = average_precision_score(y_test, p)

    results.append((name, roc, pr))
    
    print("\n" + "="*70)
    print(name)
    print("ROC AUC:", round(roc, 4))
    print("PR  AUC:", round(pr, 4))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, yhat))
    print("\nReport:\n", classification_report(y_test, yhat, digits=3))

display(pd.DataFrame(results, columns=["Model", "ROC_AUC", "PR_AUC"]).sort_values("PR_AUC", ascending=False))

# --- COST-SENSITIVE THRESHOLDING ---
print("\n--- FINANCIAL RISK OPTIMIZATION ---")
best_model_name = pd.DataFrame(results, columns=["Model", "ROC_AUC", "PR_AUC"]).sort_values("PR_AUC", ascending=False).iloc[0]['Model']
best_model = fitted[best_model_name]
probs = best_model.predict_proba(X_test)[:, 1]

# Use existing Cost Logic from Step 3
if "Est_Breach_Cost" in df_sla.columns:
    test_costs = df_sla.loc[X_test.index, "Est_Breach_Cost"].fillna(0).values
else:
    # Fallback if cost column missing (Safety)
    priority = df_sla.loc[X_test.index, "Ticket Priority"].astype(str)
    test_costs = np.where(priority.isin(["Critical", "High"]), 50, 10)

thresholds = np.linspace(0.05, 0.95, 19)
best = None

for t in thresholds:
    pred = (probs >= t).astype(int)
    # Expected loss: FN costs you the breach cost; FP costs you time/effort (set to $2 handling cost)
    FN = ((y_test.values == 1) & (pred == 0))
    FP = ((y_test.values == 0) & (pred == 1))

    expected_loss = (test_costs[FN].sum()) + (FP.sum() * 2)
    breach_recall = (pred[y_test.values == 1].mean()) 
    
    row = (t, expected_loss, breach_recall)
    if (best is None) or (expected_loss < best[1]):
        best = row

print(f"Optimal Risk Threshold={best[0]:.2f}") 
print(f"Minimizes Expected Financial Loss to ${best[1]:,.0f} (vs Default 0.50 Threshold)")
print(f"Breach Recall at this threshold: {best[2]:.2%}")

## 6. Financial Risk Evaluation
Quantifying the monetary impact of our SLA failures to justify investment.

In [ ]:
total_risk = df_sla['Est_Breach_Cost'].sum()
monthly_risk = total_risk / 3  # Assuming dataset covers ~3 months (adjust based on data)

print(f"Total Estimated Breach Cost (Historical): ${total_risk:,.2f}")
print(f"Average Monthly Financial Risk: ${monthly_risk:,.2f}")

# Breakdown by Priority
risk_breakdown = df_sla.groupby('Ticket Priority')['Est_Breach_Cost'].sum().sort_values(ascending=False)
print("\n--- Risk Concentration by Priority ---")
print(risk_breakdown)

## 7. Optimization / Simulation
Designing the "Shift Overlap" strategy to mitigate the 10 PM bottleneck.

In [ ]:
# Hourly Risk Heatmap
hourly_risk = df_sla.groupby('Hour_of_Day').agg(
    Volume=('Ticket ID', 'count'),
    Breach_Rate=('Is_SLA_Breach_Numeric', 'mean'),
    Total_Cost=('Est_Breach_Cost', 'sum')
).reset_index()

fig = px.bar(hourly_risk, x='Hour_of_Day', y='Total_Cost', 
             title='Financial Loss by Hour of Day (Where should we add staff?)',
             color='Breach_Rate', color_continuous_scale='Reds')
fig.show()

# Recommendation Logic
peak_loss_hour = hourly_risk.loc[hourly_risk['Total_Cost'].idxmax(), 'Hour_of_Day']
print(f"Recommendation: Deploy 'Overlap Shift' starting at {peak_loss_hour}:00 to mitigate peak financial loss.")

## 8. Executive Storytelling
Summarizing the findings for the Board.

In [ ]:
print("--- STRATEGIC EXECUTIVE SUMMARY ---")
print(f"1. FINANCIAL EXPOSURE: We are losing ~${monthly_risk:,.0f}/month due to SLA breaches.")
print(f"2. CRITICAL FAILURE: {risk_breakdown.index[0]} tickets account for the majority of this cost.")
print(f"3. OPERATIONAL FIX: Implementing a shift overlap at {peak_loss_hour}:00 will address the highest risk interval.")
print(f"4. AI PREDICTION: Random Forest model deployed to flag at-risk tickets with {roc_auc_score(y_test, probs):.2f} AUC accuracy.")

In [ ]:
# --- DASHBOARD EXPORT (single source of truth) ---

# Ensure we use the SLA-processed dataframe
export_df = df_sla.copy()

# 1. Ensure Ticket ID exists
if "Ticket ID" not in export_df.columns:
    export_df.insert(0, "Ticket ID", range(1, len(export_df) + 1))

# 2. create Ticket_Date for trending
export_df["Ticket_Date"] = pd.to_datetime(export_df["First Response Time"]).dt.date

# 3. Add Predictions if available (Best Effort)
if 'best_model' in locals():
    # Predict on the full dataset for the dashboard
    try:
        # Re-encode full dataset using the pipeline
        # Note: We need to match the feature set used in training
        feature_cols = ['Ticket Type', 'Ticket Priority', 'Ticket Channel', 'Product Purchased']
        X_full = export_df[feature_cols]
        
        # Use the fitted model to predict probability
        # The model is a Pipeline, so it handles preprocessing
        export_df['Pred_Breach_Prob'] = best_model.predict_proba(X_full)[:, 1]
        
        # Create Risk Buckets
        export_df["Risk_Bucket"] = pd.cut(
            export_df["Pred_Breach_Prob"],
            bins=[0, 0.3, 0.6, 1.0],
            labels=["Low Risk", "Medium Risk", "High Risk"]
        )
        print("✅ Predictive Scores added to Dashboard Export.")
    except Exception as e:
        print(f"⚠️ Could not add predictions to full export: {e}")
        export_df["Risk_Bucket"] = "N/A"
else:
    export_df["Risk_Bucket"] = "N/A"

# 4. Select Columns for Tableau
dashboard_cols = [
    "Ticket ID",
    "Ticket_Date",
    "Ticket Type",
    "Ticket Priority",
    "Ticket Channel",
    "Product Purchased",
    "Resolution_Hours",
    "SLA_Target_Hours",
    "Is_SLA_Breach",
    "Est_Breach_Cost",      # Matches our notebook's logic
    "Pred_Breach_Prob",
    "Risk_Bucket"
]

final_cols = [c for c in dashboard_cols if c in export_df.columns]
df_dashboard = export_df[final_cols].copy()

# Rename 'Est_Breach_Cost' to 'Breach_Cost' for Tableau cleanliness
if "Est_Breach_Cost" in df_dashboard.columns:
    df_dashboard.rename(columns={"Est_Breach_Cost": "Breach_Cost"}, inplace=True)

# 5. Save
out_path = "../outputs/dashboard/customer_support_sla_dashboard.csv"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
df_dashboard.to_csv(out_path, index=False)

print(f"✅ Exported: {out_path}")
display(df_dashboard.head())